<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/NEW_Prediction_thresholds_tuning_trainer_EN_11_24000__24100_ml1024_ep5_lr1e_5_th05_at1024_fp16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Error: gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Ignore this error as gcsfs is not used.

In [1]:
!pip install -q transformers datasets

import datetime
import json
import numpy as np
import os
import pandas as pd
import torch

from datasets         import DatasetDict
from google.colab     import files, userdata
from huggingface_hub  import create_repo, hf_hub_download, login, whoami
from sklearn.metrics  import classification_report
from transformers     import LongformerTokenizerFast, LongformerForSequenceClassification
from torch.utils.data import DataLoader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.9 MB/s eta 0:00:00


## Hugging Face Hub (HF Hub) authenticate

In [2]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')

login(token=hf_token)

# Check
user = whoami(token=hf_token)
assert user['name'] == 'claudelepere', f"{user['name']} is not claudelepere"

print(f"user: {user}")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


user: {'type': 'user', 'id': '66ec3d5f61228b02f8780beb', 'name': 'claudelepere', 'fullname': 'Claude Lepère', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/66ec3d5f61228b02f8780beb/gvnf9pvm2KvE90ETMUQo3.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'jobs_token', 'role': 'fineGrained', 'createdAt': '2025-01-04T17:44:35.493Z', 'fineGrained': {'canReadGatedRepos': False, 'global': [], 'scoped': [{'entity': {'_id': '66ec3d5f61228b02f8780beb', 'type': 'user', 'name': 'claudelepere'}, 'permissions': ['repo.content.read', 'repo.write']}]}}}}


## HF Hub repo

In [3]:
repo_id = 'claudelepere/jobs_EN_11_0_24000_micro_micro_epoch5_tuned_thresholds'

## Checks

In [4]:
!python -V

print(f"currentdir: {os.getcwd()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

Python 3.11.11
currentdir: /content
device: cuda


In [5]:
skills           = 11
all_rows_low     = 24000
all_rows_high    = 24100
num_datapoints   = all_rows_high - all_rows_low
max_length       = 1024
batch_size       = 4*8
threshold_tuning = True

datasetDict_zip_file_name = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}.zip"
datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]

print(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
print(f"datasetDict_dir_name     : {datasetDict_dir_name}")
print()

#revision_name = "EN_11_0_24000_ml1024_ep5_lr1e-05_th0.5_at1024_fp16_ba8x4"
#print(f"revision_name: {revision_name}")

datasetDict_zip_file_name: dataset_EN_11_24000_24100.zip
datasetDict_dir_name     : dataset_EN_11_24000_24100



## Out Of Memory (OOM)

### OOM: check for and kill zombie processes

In [6]:
!ps aux | grep python
!kill -9 <PID>
if torch.cuda.is_available():
    !nvidia-smi
    print(torch.cuda.memory_summary())

root          87  1.5  0.0      0     0 ?        Z    18:44   0:13 [python3] <defunct>
root          88  0.0  0.0  63772 50256 ?        S    18:44   0:00 python3 /usr/local/bin/colab-file
root         133  0.4  0.1 779756 127152 ?       Sl   18:44   0:03 /usr/bin/python3 /usr/local/bin/j
root        3462 56.8  1.4 12139024 1243096 ?    Ssl  18:58   0:16 /usr/bin/python3 -m colab_kernel_
root        3504  1.9  0.0 545168 20492 ?        Sl   18:58   0:00 /usr/bin/python3 /usr/local/lib/p
root        3724  0.0  0.0   7376  3424 ?        S    18:58   0:00 /bin/bash -c ps aux | grep python
root        3726  0.0  0.0   6484  2404 ?        S    18:58   0:00 grep python
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `kill -9 <PID>'
Tue Mar  4 18:58:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----

### OOM: free up GPU memory

In [7]:
torch.cuda.empty_cache()

## Upload and unzip job dataset

In [8]:
def upload_unzip_dataset(file_name=datasetDict_zip_file_name):
    # Check if the file exists
    if not os.path.exists(file_name):
        print(f"'{file_name}' not found in /content. Uploading...")

        uploaded_files = files.upload()                              # Prompt file upload dialog

        if file_name not in uploaded_files:
            raise FileNotFoundError(f"'{file_name}' not found.")
        print(f"'{file_name}' successfully uploaded to /content")

        uploaded_file_name = list(uploaded_files.keys())[0]          # Get the name of the uploaded file

        !unzip {uploaded_file_name}

        unzipped_dir_name = os.path.splitext(uploaded_file_name)[0]
        assert unzipped_dir_name==datasetDict_dir_name, "unzipped_dir_name != datasetDict_dir_name"
    else:
        print(f"'{file_name}' already exists in /content.")

upload_unzip_dataset(datasetDict_zip_file_name)

'dataset_EN_11_24000_24100.zip' not found in /content. Uploading...


Saving dataset_EN_11_24000_24100.zip to dataset_EN_11_24000_24100.zip
'dataset_EN_11_24000_24100.zip' successfully uploaded to /content
Archive:  dataset_EN_11_24000_24100.zip
  inflating: dataset_EN_11_24000_24100/dataset_dict.json  
  inflating: dataset_EN_11_24000_24100/inference/data-00000-of-00001.arrow  
  inflating: dataset_EN_11_24000_24100/inference/dataset_info.json  
  inflating: dataset_EN_11_24000_24100/inference/state.json  


## Create datasetDict (HF DatasetDict) = 1 HF Dataset, inference
The sequence of datapoints of dataset=datasetDict['inference'] is the same as the sequence of SELECT, id, ... WHERE in_LanguageId=1 ORDER BY id DESC in MySQL.

In [9]:
datasetDict = DatasetDict.load_from_disk(datasetDict_dir_name)
dataset     = datasetDict['inference']

print(f"dataset: {type(dataset)} shape={dataset.shape}\n{dataset}")     # <class 'datasets.arrow_dataset.Dataset'> shape=(100, 8)
print(f"dataset.features: {type(dataset.features)} shape={dataset.features}\n{dataset.features}")

# Convert the dataset to a pandas dataframe
df_original = pd.DataFrame(dataset)
print(f"df_original: {type(df_original)} shape={df_original.shape}\n{df_original}")                         # <class 'pandas.core.frame.DataFrame'> shape=(100, 8)


dataset: <class 'datasets.arrow_dataset.Dataset'> shape=(100, 8)
Dataset({
    features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
    num_rows: 100
})
dataset.features: <class 'datasets.features.features.Features'> shape={'id': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), '390': Value(dtype='bool', id=None), '135': Value(dtype='bool', id=None), '136': Value(dtype='bool', id=None), '137': Value(dtype='bool', id=None), '138': Value(dtype='bool', id=None), '139': Value(dtype='bool', id=None)}
{'id': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), '390': Value(dtype='bool', id=None), '135': Value(dtype='bool', id=None), '136': Value(dtype='bool', id=None), '137': Value(dtype='bool', id=None), '138': Value(dtype='bool', id=None), '139': Value(dtype='bool', id=None)}
df_original: <class 'pandas.core.frame.DataFrame'> shape=(100, 8)
       id                                               text    390    135  \
0   99388  Consort

## Create labels (list), id2label (dict) and label2id (dict).
**The sequence of the labels list is the same as in dataset.
And the sequences of the optimized thresholds, true labels and predictions are preserved.**

* dataset 7_1000_125_125  ,  48 labels
* dataset 7_128_18_54     ,  42 labels
* dataset 8910_1087_68_204, 206 labels
* dataset 11_1000         ,   6 labels

In [10]:
labels = [label for label in dataset.features.keys() if label not in ['id', 'text']]
#labels.sort()

print(f"labels: {type(labels)} {len(labels)}\n{labels}")

num_labels = len(labels)

labels: <class 'list'> 6
['390', '135', '136', '137', '138', '139']


In [11]:
id2label = {idx: label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

id2label: <class 'dict'> 6
{0: '390', 1: '135', 2: '136', 3: '137', 4: '138', 5: '139'}


In [12]:
label2id = {label: idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

label2id: <class 'dict'> 6
{'390': 0, '135': 1, '136': 2, '137': 3, '138': 4, '139': 5}


## Download the tokenizer and the model

In [13]:
print("Tokenizer")
tokenizer = LongformerTokenizerFast.from_pretrained(repo_id, timeout=60)  # Increased timeout to 60 seconds)

print("Model")
model = LongformerForSequenceClassification.from_pretrained(repo_id)
model.eval()  # Ensures no gradient computation

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Tokenizer


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Model


config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
          

## Downloads the optimized thresholds

In [14]:
file_path = hf_hub_download(repo_id=repo_id, filename="optimized_thresholds.json", cache_dir="./my_HF_downloads")
print(f"file_path: {file_path}")

# Load the JSON file
with open(file_path, 'r') as json_file:
    optimized_thresholds_json = json.load(json_file)

#print(f"optimized_thresholds_json: {type(optimized_thresholds_json)} {len(optimized_thresholds_json)}")
#print(json.dumps(optimized_thresholds_json, indent=2))

optimized_thresholds = np.array(list(optimized_thresholds_json.values()))
print(f"optimized_thresholds: {type(optimized_thresholds)} shape={optimized_thresholds.shape}\n{optimized_thresholds}")


optimized_thresholds.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

file_path: ./my_HF_downloads/models--claudelepere--jobs_EN_11_0_24000_micro_micro_epoch5_tuned_thresholds/snapshots/5c82e02d6f7e012adf88db5de182bb24938aac94/optimized_thresholds.json
optimized_thresholds: <class 'numpy.ndarray'> shape=(6,)
[0.2  0.4  0.35 0.35 0.55 0.45]


In [15]:
# Step 1: Tokenize dataset
def tokenize_function(examples):
  return tokenizer(examples["text"], padding='max_length', max_length=max_length, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
# <class 'datasets.arrow_dataset.Dataset'> shape=(100, 10)
print(f"tokenized_dataset: {type(tokenized_dataset)} shape={tokenized_dataset.shape}\n{tokenized_dataset}")

tokenized_df = tokenized_dataset.to_pandas()
print(f"tokenized_df: {type(tokenized_df)} shape={tokenized_df.shape}\n{tokenized_df}")

# Step 2: Convert to PyTorch DataLoader
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"] + labels)
dataloader = DataLoader(tokenized_dataset, batch_size=batch_size)

# Step 3: Run Prediction
all_preds = []

with torch.no_grad():                       # No gradients needed for prediction
    for batch in dataloader:
        # Move batch to GPU if available
        inputs = {k: batch[k].to(device) for k in ["input_ids", "attention_mask"]}

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits  # Model outputs raw logits

        # Apply sigmoid to convert logits into probabilities
        probs = torch.sigmoid(logits)

        # Move probs to CPU and convert to NumPy
        all_preds.extend(probs.cpu().numpy())

all_preds_arr = np.array(all_preds)
print(f"all_preds    : {type(all_preds)} len={len(all_preds)}\n{all_preds}")
print(f"all_preds_arr: {type(all_preds_arr)} shape={all_preds_arr.shape}\n{all_preds_arr}")

# Step 4: Convert Probabilities to Binary Predictions
if threshold_tuning:
    thresholds = optimized_thresholds
else:
    thresholds = np.full(num_labels, 0.5)
print(f"thresholds: {type(thresholds)} shape={thresholds.shape}\n{thresholds}")

binary_preds = (np.array(all_preds) > thresholds).astype(int)   # <class 'numpy.ndarray'> shape=(100, 6)
print(f"binary_preds: {type(binary_preds)} shape={binary_preds.shape}\n{binary_preds}")

# Step 5: Compare with True Labels
true_labels = tokenized_df[labels].values                       # <class 'numpy.ndarray'> shape=(100, 6)
print(f"true_labels: {type(true_labels)} shape={true_labels.shape}\n{true_labels}")  # Should match binary_preds.shape

print(f"classification_report:\n{classification_report(true_labels, binary_preds, zero_division=0)}")

relevant_columns = ['id', '390', '135', '136', '137', '138', '139']
df_ZZZ = df_original[relevant_columns].copy()
df_ZZZ['true_labels'] = true_labels.tolist()
df_ZZZ['preds'] = binary_preds.tolist()
print(f"df_ZZZ: {type(df_ZZZ)} shape={df_ZZZ.shape}\n{df_ZZZ}")


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Initializing global attention on CLS token...


tokenized_dataset: <class 'datasets.arrow_dataset.Dataset'> shape=(100, 10)
Dataset({
    features: ['id', 'text', '390', '135', '136', '137', '138', '139', 'input_ids', 'attention_mask'],
    num_rows: 100
})
tokenized_df: <class 'pandas.core.frame.DataFrame'> shape=(100, 10)
       id                                               text    390    135  \
0   99388  Consort NT - Second Line Engineer (Windows Ser...  False  False   
1   99387  MyNextCompany - IT Infrastructure Project Mana...  False  False   
2   99386  Consort NT - Support Engineer (Windows Server,...  False  False   
3   99384  Visser & Van Baars - Market- & Business Intell...  False  False   
4   99383  Computer Recruitment Services - Change Manager...  False  False   
..    ...                                                ...    ...    ...   
95  99155  Koalect - Front-end Developer   HTML5, Javascr...  False  False   
96  99152  MyNextCompany - Junior Digital Project Officer...   True   True   
97  99151  Widetech 

In [16]:
raise Exception("I stop here")

Exception: I stop here

batch_size = 8

(100, 6)

              precision    recall  f1-score   support

           0       0.80      0.67      0.73         6
           1       0.92      0.55      0.69        22
           2       0.88      0.83      0.85        70
           3       0.94      0.99      0.96        90
           4       0.90      0.94      0.92        84
           5       0.67      0.33      0.44        12

   micro avg       0.90      0.87      0.88       284

   macro avg       0.85      0.72      0.77       284

weighted avg       0.90      0.87      0.87       284

 samples avg       0.91      0.89      0.89       284

batch_size = 4 x 8

(100, 6)
              precision    recall  f1-score   support

           0       0.80      0.67      0.73         6
           1       0.92      0.55      0.69        22
           2       0.88      0.83      0.85        70
           3       0.94      0.99      0.96        90
           4       0.90      0.94      0.92        84
           5       0.67      0.33      0.44        12

   micro avg       0.90      0.87      0.88       284

   macro avg       0.85      0.72      0.77       284

weighted avg       0.90      0.87      0.87       284

 samples avg       0.91      0.89      0.89       284


 With tuned thresholds, batch_size = 4 x 8

 Predictions Shape: (100, 6)
Thresholds Shape: (6,)
(100, 6)
              precision    recall  f1-score   support

           0       0.80      0.67      0.73         6
           1       0.70      0.64      0.67        22
           2       0.85      0.94      0.89        70
           3       0.93      0.99      0.96        90
           4       0.90      0.98      0.94        84
           5       0.67      0.33      0.44        12

   micro avg       0.88      0.91      0.89       284

   macro avg       0.81      0.76      0.77       284

weighted avg       0.87      0.91      0.89       284

 samples avg       0.89      0.93      0.90       284

# Making predictions

In [ ]:
dataLoader = DataLoader(tokenized_dataset, batch_size=batch_size, collate_fn=lambda x: {k: torch.cat([d[k] for d in x], dim=0) for k in x[0]})

all_preds = []

for batch in dataLoader:
  # Move tensors to the device if available
  batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items() if k in ["input_ids", "attention_mask"]}
  # Filter out unwanted keys like 'id'

  with torch.no_grad():
    outputs = model(**batch) # Pass the filtered batch to the model
    logits = outputs.logits
    preds = torch.sigmoid(logits).cpu().numpy()
    all_preds.append(preds)

threshold    = 0.5
binary_preds = (np.array(all_preds) > threshold).astype(int)
true_labels  = np.array(dataset['labels'])

# Compute classification report
#from sklearn.metrics import classification_report
print(classification_report(true_labels, binary_preds, zero_division=0))


In [ ]:
print("It's the end")

In [ ]:
raise Exception("I stop here")

In [ ]:
print(datasetDict['inference'])
print(datasetDict['inference']['labels'])

In [ ]:
with torch.no_grad():  # Disable gradient calculation during inference
    outputs = model(
        input_ids=inputs.input_ids.to(device),
        attention_mask=inputs.attention_mask.to(device)
    )


In [ ]:
probs = torch.sigmoid(outputs.logits)


In [ ]:
best_thresholds = [0.4, 0.3, 0.3, 0.3, 0.4, 0.5]
preds = np.zeros_like(probs)  # Initialize predictions array
for label_idx in range(num_labels):
  preds[:, label_idx] = (probs[:, label_idx] > best_thresholds[label_idx])  #.astype(int)

In [ ]:
print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")
print(f"preds: {type(preds)} shape={preds.shape}\n{preds}")

In [ ]:
print(f"labels: {type(labels)} {len(labels)}\n{labels}")
# '390': False, '135': False, '136': True, '137': True, '138': True, '139': False}